In [298]:
import pandas as pd
import numpy as np
from bokeh.io import show
from bokeh.models import Button, CustomJS, Select, Slider, Toggle
from bokeh.layouts import column, row
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, DataRange1d, Select, CheckboxButtonGroup, TextInput, MultiChoice,TableColumn,widgets, DataTable
from bokeh.palettes import Blues4
from bokeh.plotting import figure

In [299]:
df = pd.read_csv("./raw_data.csv",encoding="utf-8")

In [300]:
# generate player list
df2 = df.drop_duplicates()
df2 = pd.concat([df2["player_win"],df2["player_lose"]]).drop_duplicates()
print(len(players))

318


In [301]:
# geenrate dim table
# df2 = df.drop_duplicates()
# leader = pd.concat([df2["wla"],df2["wl1"],df2["wl2"],df2["wl3"]]).drop_duplicates()
# wonder = pd.concat([df2["ww1"],df2["ww2"],df2["ww3"],df2["ww4"],df2["ww5"],df2["ww6"]]).drop_duplicates()
# leader = leader[leader!='0'].rename("leader")
# wonder = wonder[wonder!='0'].rename("wonder")
# leader.to_csv('./leader.csv',index=False)
# wonder.to_csv('./wonder.csv',index=False)

## Player Win Rate

In [302]:
leader = pd.read_csv("./leader.csv",encoding="utf-8",index_col="leader")
wonder = pd.read_csv("./wonder.csv",encoding="utf-8",index_col="wonder")

In [303]:
def stdResult(df):
    df = df.dropna().sort_values(by=["age","winrate","card"],ascending=[True,False,True],key=lambda x:x.map({'A':0,'1':1,'2':2,'3':3}))
    df["winrate"] = df["winrate"].round(2)
    df.columns=["时代","领袖","胜利","失败","胜率"]
    return df


In [304]:
def getLeaderResult(id):
    res = pd.DataFrame(columns=["age","card","win","lose","winrate"])
    df_win = df[df["player_win"].isin(id)]
    df_lose = df[df["player_lose"].isin(id)]
    for i,v in leader.iterrows():
        win = df_win[df_win["wl"+v[0]]==i].shape[0]
        lose = df_lose[df_lose["ll"+v[0]]==i].shape[0]
        res.loc[res.shape[0]] = [v[0],i,win,lose,win/(win+lose) if win+lose>0 else np.nan]
    return stdResult(res)

In [305]:
def getWonderResult(id):
    res = pd.DataFrame(columns=["age","card","win","lose","winrate"])
    df_win = df[df["player_win"].isin(id)]
    df_lose = df[df["player_lose"].isin(id)]
    for i,v in wonder.iterrows():
        win = df_win[(df_win["ww1"]==i)|(df_win["ww2"]==i)|(df_win["ww3"]==i)|(df_win["ww4"]==i)|(df_win["ww5"]==i)|(df_win["ww6"]==i)|(df_win["ww7"]==i)|(df_win["ww8"]==i)].shape[0]
        lose = df_lose[(df_lose["lw1"]==i)|(df_lose["lw2"]==i)|(df_lose["lw3"]==i)|(df_lose["lw4"]==i)|(df_lose["lw5"]==i)|(df_lose["lw6"]==i)|(df_lose["lw7"]==i)|(df_lose["lw8"]==i)].shape[0]
        res.loc[res.shape[0]] =  [v[0],i,win,lose,win/(win+lose) if win+lose>0 else np.nan]
    return stdResult(res)

In [306]:
df2 = getWonderResult(["Ender_Wiggin"])
source = ColumnDataSource(df2)
columns = [TableColumn(field=Ci, title=Ci) for Ci in df2.columns]
leader_table = DataTable(columns=columns, source=source)


In [307]:
def selectUser(attr,new,old):
    user_df = getLeaderResult(new)
    leader_table = DataTable(columns=columns, source=ColumnDataSource(user_df))

In [310]:
leaders = ["Age A Leader","Age I Leader","Age II Leader","Age III Leader"]
leader_group = CheckboxButtonGroup(labels=leaders, active=[1,1,1,1])
leader_group.js_on_click(CustomJS(code="""
    console.log('leader_group: active=' + this.active, this.toString())
"""))
user = MultiChoice(value=[],options=players)
user.on_change("value",selectUser)
widg = column(leader_group, user, leader_table)

show(widg)

ValueError: not all callback values are CustomJS instances

In [ ]:
# curdoc().add_root(widg)

In [ ]:
'''
import import_ipynb
from cov_leader import *
from cov_wonder import *

txt = open("5239.txt", "rt", encoding="utf-16")
s = txt.readlines()
txt.close()

txt = open("conbo.txt", "wt", encoding="utf-16")
for i in range(48+32):
    if i < 48:
        t1 = cov_num_to_leader(i)
    else:
        t1 = cov_num_to_wonder(i-48)
    for j in range(48+32):
        if j < 48:
            t2 = cov_num_to_leader(j)
        else:
            t2 = cov_num_to_wonder(j-48)
        leader_wonder = [t1, t2]
        txt.write(leader_wonder[0] + " " + leader_wonder[1] + " ")
        win = 0
        lose = 0
        for k in range(1, len(s)):
            s_s = s[k].split()
            if leader_wonder[0] in s_s[5:9] or leader_wonder[0] in s_s[13:21]:
                if leader_wonder[1] in s_s[5:9] or leader_wonder[1] in s_s[13:21]:
                    win += 1
            elif leader_wonder[0] in s_s[9:13] or leader_wonder[0] in s_s[21:]:
                if leader_wonder[1] in s_s[9:13] or leader_wonder[1] in s_s[21:]:
                    lose += 1

        txt.write(str(win) + " " + str(lose)+"\n")
'''

importing Jupyter notebook from cov_leader.ipynb
importing Jupyter notebook from cov_wonder.ipynb
